In [1]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

**Đến trang chủ**

In [3]:
url = 'https://vnexpress.net'
driver.get(url)

driver.execute_script(
    'window.scrollTo({top: document.body.scrollHeight, behavior: "smooth"})')
time.sleep(5)
driver.execute_script('window.scrollTo({top: 0, behavior: "smooth"})')

**Lấy link đến những bài viết**

In [4]:
POST_LINK_CLS = '.title-news a:not(.count_cmt)'

post_links_el = driver.find_elements(By.CSS_SELECTOR, POST_LINK_CLS)
post_links = [
    el.get_attribute('href')
    for el in post_links_el]
print('Found', len(post_links), 'links.')
post_links

Found 103 links.


['https://vnexpress.net/mua-lu-tram-nam-co-mot-tan-pha-hong-kong-4651231.html',
 'https://vnexpress.net/doi-tac-my-4651516.html',
 'https://vnexpress.net/tong-thong-my-roi-an-do-len-duong-toi-viet-nam-4651225.html',
 'https://vnexpress.net/lao-dong-mat-viec-duoc-vay-von-lai-suat-thap-de-lam-an-4651517.html',
 'https://vnexpress.net/cau-thu-yemen-khoc-oa-khi-thua-viet-nam-4651501.html',
 'https://vnexpress.net/xuong-so-che-day-chuoi-o-u-minh-thuong-4649029.html',
 'https://vnexpress.net/anh-tho-toi-khong-kiem-duoc-nguoi-dan-ong-hon-minh-4651160.html',
 'https://vnexpress.net/chay-hai-cua-hang-o-dong-nai-4651498.html',
 'https://vnexpress.net/nguoi-chet-trong-dong-dat-morocco-len-hon-2-000-4651379.html',
 'https://vnexpress.net/pham-tien-san-vo-dich-vnexpress-marathon-ha-long-2023-4651355-tong-thuat.html',
 'https://vnexpress.net/double2t-toi-se-bo-nghe-cat-toc-de-theo-am-nhac-4651367.html',
 'https://vnexpress.net/thieu-nu-16-tuoi-bi-chu-hanh-ha-4651499.html',
 'https://vnexpress.net/xu

> Nhận xét: link của các post có những link không ở trang vnexpress, cần xử lý thêm.

In [5]:
valid_post_links = []
VNE_REGEX = '^https://vnexpress.net'
for post_link in post_links:
    if re.match(VNE_REGEX, post_link):
        valid_post_links.append(post_link)
    else:
        print('Invalid link:', post_link)

print('Total valid links:', len(valid_post_links))

Invalid link: https://c.eclick.vn/r?beacon=zizlzqznzgzgzkzjzjzqzozizjzhzqzrzozlzlzkzjzozizgzizkzr2pzhzjzjzjzjzgzkzlzrzq2pzhzjzjzjzhzjznzmzizgzozizizlzdzizizjzdznzgzdzhzhzhzo1u1x2520zj29zh2e2e2j1x2f1yzg1v2hzdzizlzqznzgzgzkzjzjzrzd21202a&link=https%3A%2F%2Fc.eclick.vn%2Fr%2F2000204513%2F10298%2Fzizlzqznzgzgzkzjzjzqzozizjzhzqzrzozhzjzjzjzhzjznzmzizgzozizizlzdzizizjzdznzgzdzhzhzhzo1u1x2520zj29zh2e2e2j1x2f1yzg1v2hzdzizlzqznzgzgzkzjzjzrzd21202a%2Fhttps%3A%2F%2Fyhoc.co%2Fngo-ngang-khi-bien-chung-benh-tieu-duong-duoc-day-lui-mot-cach-don-gian%3Futm_source%3DNative4915%26utm_medium%3Dngo-ngang-khi-bien-chung-benh-tieu-duong-duoc-day-lui-mot-cach-don-gian%26utm_campaign%3Dngo-ngang-khi-bien-chung-benh-tieu-duong-duoc-day-lui-mot-cach-don-gian&algo=40&cid=2000037689&cat=1028,1072,1074&bid=2000204513&gid=56&ts=1694337009&url=https%3A%2F%2Fvnexpress.net%2F&urlref=https://vnexpress.net/&hostname=vnexpress.net&tsv=1694337009&res=1512x982&zone_format=502&loc=29-1-vn&lp=29&lz=1&lc=vn&fosp_aid=bghe0l2ww

**Đi đến tất cả các trang và lấy tiêu đề + nội dung + hình thumbnail**

Yêu cầu: tiêu đề và hình

In [6]:
import os
import re
from urllib import request
from tqdm import tqdm
from string import punctuation

In [ ]:
POST_TITLE_CLS = '.sidebar-1 .title-detail'
POST_CONTENT_CLS = 'article.fck_detail p.Normal'
POST_IMG_CLS = '.fig-picture img[itemprop="contentUrl"]'
IMAGE_REGEX = r'([\w-]+(?:.jpg|.png|.webm))'

for post_link in tqdm(valid_post_links):
    # Go to that link.
    driver.get(post_link)
    time.sleep(2)
    # Get the things we need.
    # Post title.
    title = driver.find_element(By.CSS_SELECTOR, POST_TITLE_CLS)
    # Post content.
    contents_el = driver.find_elements(By.CSS_SELECTOR, POST_CONTENT_CLS)
    contents = [content_el.get_attribute(
        'innerText') for content_el in contents_el]
    # Post images.
    images_el = driver.find_elements(By.CSS_SELECTOR, POST_IMG_CLS)
    images_src = [image_el.get_attribute('src') for image_el in images_el]
    # Save images to local.
    image_file_path = '/Users/minhquang/Desktop/ml/ml-teach/lesson_5/img'

    for image_src in images_src:
        image_file_name = re.search(IMAGE_REGEX, image_src)
        if image_file_name:
            image_file_name = image_file_name.group()
            request.urlretrieve(image_src, os.path.join(
                image_file_path, image_file_name))
            print(f'Saved', image_file_name)

    break

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:05<?, ?it/s]

Saved PM-PhamMInhChinh-President-Bid-6182-4763-1694308522.jpg
Saved Bill-Clinton-toi-Vietnam001-57-2756-8446-1694284198.jpg


In [7]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [9]:
POST_TITLE_CLS = '.title-detail'
POST_CONTENT_CLS = 'article.fck_detail p.Normal'
POST_IMG_CLS = 'picture img[itemprop="contentUrl"]'
IMAGE_URL_REGEX = r'https:\/\/[\w\-\./]+(?:.jpg|.png|.webm)'
IMAGE_REGEX = r'([\w-]+(?:.jpg|.png|.webm))'


def title_to_slug(title):
    # Remove punctuations.
    title_ = re.sub(rf'[{punctuation}]', '', title)
    # Replace duplicated spaces with one space.
    title_ = re.sub(r'\s+', ' ', title_)
    title_slug = '-'.join(title_.lower().split(' '))
    return title_slug


for post_link in valid_post_links:
    # Go to that link.
    driver.get(post_link)
    driver.maximize_window()
    driver.implicitly_wait(5)

    # Scroll to end of page.
    driver.execute_script(
        'window.scrollTo({top: document.body.scrollHeight, behavior: "smooth"})')
    # driver.execute_script('window.scrollTo({top: 0, behavior: "smooth"})')

    # Get the things we need.
    # Post title.
    title = driver.find_element(
        By.CSS_SELECTOR, POST_TITLE_CLS).get_attribute('innerText')
    title_slug = title_to_slug(title)

    # Post content.
    contents_el = driver.find_elements(By.CSS_SELECTOR, POST_CONTENT_CLS)
    contents = [content_el.get_attribute(
        'innerText') for content_el in contents_el]

    # Post images.
    images_el = driver.find_elements(By.CSS_SELECTOR, POST_IMG_CLS)
    # images_el = [image for image in WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, POST_IMG_CLS)))]
    # driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', images_el[-1])
    images_src = list(set([image_el.get_attribute('src')
                      for image_el in images_el]))
    print('Found', len(images_src), 'images.')
    # Save images to local.
    image_file_path = '/Users/minhquang/Desktop/ml/ml-teach/lesson_5/img'

    for i, image_src in enumerate(images_src):
        image_file_name = re.search(IMAGE_REGEX, image_src)
        if image_file_name:
            image_file_name = image_file_name.group()
            image_save_path = os.path.join(
                image_file_path, image_file_name)
            print(f'Saving', image_src)
            _, res = request.urlretrieve(image_src, image_save_path)
            # Get image format (png, jpg, ...).
            image_type = res.get_content_subtype()
            # Rename the saved file to correct format.
            new_image_save_path = os.path.join(
                image_file_path, title_slug + '-' + str(i) + f'.{image_type}')
            os.rename(image_save_path, new_image_save_path)

Found 4 images.
Saving https://i1-vnexpress.vnecdn.net/2023/09/09/33UK8ZY-highres-1694236553.jpg?w=220&h=0&q=80&dpr=1&fit=crop&s=nNi170KWr32ctwF115yl8Q
Saving https://i1-vnexpress.vnecdn.net/2023/09/09/33UK8ZV-highres-1694237590.jpg?w=1200&h=0&q=100&dpr=1&fit=crop&s=3K0zQaHeok7-n4qWJth-7w
Saving https://i1-vnexpress.vnecdn.net/2023/09/09/33UL9JA-highres-1694236548.jpg?w=220&h=0&q=80&dpr=1&fit=crop&s=uANb12dGt_x9QaEhDBYafA


In [18]:
from multiprocessing import Pool

# TODO: implement save image using multiprocessing starmap.


def save_image_multi(image_src, index, title_slug):
    image_file_name = re.search(IMAGE_REGEX, image_src)

    if image_file_name:
        image_file_name = image_file_name.group()
        image_save_path = os.path.join(
            image_file_path, image_file_name)
        print(f'Saving', image_src)
        _, res = request.urlretrieve(image_src, image_save_path)
        # Get image format (png, jpg, ...).
        image_type = res.get_content_subtype()
        # Rename the saved file to correct format.
        new_image_save_path = os.path.join(
            image_file_path, title_slug + '-' + str(index) + f'.{image_type}')
        os.rename(image_save_path, new_image_save_path)